In [1]:
ENVIRONMENT = None # must set to "local" or "remote" before running
# ask for environment via input
if ENVIRONMENT is None:
    ENVIRONMENT = input("Please enter environment (local/remote): ")
    assert ENVIRONMENT in ["local", "remote"], "Invalid environment"

import os
import shutil
from datetime import datetime, timedelta
import urllib
import zipfile
import matplotlib.pyplot as plt
import requests
import pandas as pd

#from configs import EXT_DATA_DIR, FRD_USER_ID, FRD_DATA_DIR
# configs



if ENVIRONMENT == "local":
    if os.name == "nt":
        PROJECT_DIR = "C:\\Users\\regin\\Dropbox\\ibis"
        FRD_DATA_DIR = "E:\\frd-historical\\data\\"
        FRD_USER_ID_FP = "C:\\Users\\regin\\Dropbox\\API_KEYS\\FRD-USER-ID"
    else:
        PROJECT_DIR = "/home/reggie/Dropbox/ibis/"
        FRD_DATA_DIR = "/media/reggie/reg_ext/frd-historical/data/"
        FRD_USER_ID_FP = "/home/reggie/Dropbox/API_KEYS/FRD-USER-ID"
elif ENVIRONMENT == "remote":
    PROJECT_DIR = "/home/ubuntu/ibis/"
    FRD_DATA_DIR = "/home/ubuntu/ibis/data/frd-historical/"
    FRD_USER_ID_FP = "/home/ubuntu/FRD-USER-ID"

with open(FRD_USER_ID_FP, "r") as file:
    FRD_USER_ID = file.read().strip()
    
DATA_DIR = os.path.join(PROJECT_DIR, "data")
#FRD_DATA_DIR = os.path.join(DATA_DIR, "frd-historical")

# Trading calendar, holidays
calendar = {
    "july4":["2023-07-03"],
    "911":["2023-09-11"],
    "thanksgiving":["2023-11-24"],
    "xmas":["2023-12-25"],
    "nyd":["2023-01-01"],
}

timeframes = ["1min", "5min", "daily", "weekly", "monthly"]
ZIP_FILES_DIR = f"/media/reggie/reg_ext/frd-historical/data/stock/{timeframe}/zips/"
CSV_DEST_DIR = f"/media/reggie/reg_ext/frd-historical/data/stock/{timeframe}/csv/"

print(FRD_DATA_DIR, os.listdir(FRD_DATA_DIR))
print(DATA_DIR, os.listdir(DATA_DIR))

/home/ubuntu/ibis/data/frd-historical/ ['First Rate Data Stock DB Schema.txt', 'api-documentation']
/home/ubuntu/ibis/data ['README.md', 'frd-historical']


In [ ]:
def extract_zip(src_path, dest_dir):
    """
    Extract a .zip file to a directory.
    Parameters
    ----------
    src_path : string
        Path to the .zip file to extract.
    
    Returns
    -------
    dest_dir : string
        Path to the directory where the .txt files were extracted.
    """
    #dest_dir = os.path.join(FRD_DATA_DIR, "tmp")
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    print(f"Extracting {src_path} to {dest_dir}")
    with zipfile.ZipFile(src_path, 'r') as zip_ref:
        zip_ref.extractall(path=dest_dir)
    n_files = len(os.listdir(dest_dir))
    print(f"Extracted {n_files} files from {src_path}")
    return dest_dir

def extract_stock_zips(**params):
    """Extract all .zip files for the given parameters."""
    # Unpack parameters
    adjustment = params['adjustment']
    timeframe = params['timeframe']
    ticker_first_letter = params['ticker_first_letter']
    
    ZIP_FILES_DIR = f"/media/reggie/reg_ext/frd-historical/data/stock/{timeframe}/zips/"
    CSV_DEST_DIR = f"/media/reggie/reg_ext/frd-historical/data/stock/{timeframe}/csv/"

    # Find zip files
    print(f"Found {len(os.listdir(ZIP_FILES_DIR))} zip files in {ZIP_FILES_DIR}")
    zip_path = os.path.join(ZIP_FILES_DIR, f"{ticker_first_letter}_{adjustment}_{timeframe}.zip")
    print(CSV_DEST_DIR)
    if not os.path.isdir(CSV_DEST_DIR):
        os.makedirs(CSV_DEST_DIR)
        print(f"Created directory {CSV_DEST_DIR}")
    
    # Extract
    src_path = extract_zip(src_path=zip_path, dest_dir=CSV_DEST_DIR)
    print(src_path)

    files = [f for f in os.listdir(src_path) if f.endswith('.txt')]
    print(f"Number of files: {len(files)} in {src_path}")
    return 0

In [4]:
import requests
import zipfile
import os
from io import BytesIO

class AssetPriceDownloader:
    def __init__(self):
        self.base_url = "https://firstratedata.com/api/data_file"
        self.userid = FRD_USER_ID
        self.FRD_DATA_DIR = "/home/ubuntu/ibis/data/frd-historical/"

    def download_data(self, asset_type, period, timeframe, adjustment):
        params = {
            "type": asset_type,
            "period": period,
            "timeframe": timeframe,
            "adjustment": adjustment,
            "userid": self.userid
        }
        response = requests.get(self.base_url, params=params)
        if response.status_code == 200:
            fp = os.path.join(self.FRD_DATA_DIR, f"{asset_type}_{period}_{timeframe}_{adjustment}.zip")
            self._save_zip(response.content, fp)
        else:
            print(f"Failed to download data: {response.status_code}")

    def _save_zip(self, content, filename):
        with open(filename, 'wb') as file:
            file.write(content)
        self._extract_zip(filename)

    def _extract_zip(self, filepath):
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            zip_ref.extractall(os.path.splitext(filepath)[0])
        os.remove(filepath)


In [5]:

# Create an instance of the downloader
downloader = AssetPriceDownloader(USER_ID)

# Define parameters
asset_type = "stock"        # Example values: stock, etf, futures, crypto, index, fx
period = "week"             # Example values: full, month, week, day
timeframe = "1day"          # Example values: 1min, 5min, 30min, 1hour, 1day
adjustment = "adj_split"    # Example values: adj_split, adj_splitdiv, UNADJUSTED

# Download data
downloader.download_data(asset_type, period, timeframe, adjustment)

In [17]:
CSV_DIR = os.path.join(FRD_DATA_DIR, asset_type, "csv", timeframe, adjustment)

print(f"Directory for csv files: {CSV_DIR}")
print(f"Directory for zip files: {ZIPS_DIR}")

# make dirs
if not os.path.exists(CSV_DIR):
    os.makedirs(CSV_DIR)
if not os.path.exists(ZIPS_DIR):
    os.makedirs(ZIPS_DIR)

print(f"CSV directory has {len(os.listdir(CSV_DIR))} files")
print(f"ZIP directory has {len(os.listdir(ZIPS_DIR))} files")

letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
for ticker_first_letter in letters:
    ticker_csv_dir = os.path.join(CSV_DIR, ticker_first_letter)
    if not os.path.exists(ticker_csv_dir):
        os.makedirs(ticker_csv_dir)

Directory for csv files: /home/ubuntu/ibis/data/frd-historical/stock/csv/1min/adj_splitdiv
Directory for zip files: /home/ubuntu/ibis/data/frd-historical/stock/zips/1min/adj_splitdiv
CSV directory has 0 files
ZIP directory has 0 files


In [72]:

from tqdm.notebook import tqdm

def download_frd_data(params, zips_dir, overwrite=False):
    ticker_range = params['ticker_range']
    period = params['period']
    adjustment = params['adjustment']
    timeframe = params['timeframe']
    zip_fp = os.path.join(zips_dir, f"{ticker_range}_{period}_{adjustment}_{timeframe}.zip")

    if os.path.exists(zip_fp) and not overwrite:
        print(f"File already exists: {zip_fp}")
        return
    
    base_url = "https://firstratedata.com/api/data_file"
    
    response = requests.get(base_url, params=params)
    print(response.url)
    if response.status_code == 200:      
        with open(zip_fp, 'wb') as file:
            file.write(response.content)
        print(f"ZIP file saved: {zip_fp}")
    else:
        print(f"Failed to download data: {response.status_code}")

def extract_frd_zip(src_zips_dir, dest_csv_dir, ticker_first_letter, period='full', adjustment='adj_splitdiv', timeframe='1min', overwrite=False):
    src_zip_fp = os.path.join(src_zips_dir, f"{ticker_first_letter}_{period}_{adjustment}_{timeframe}.zip")
    dest_csv_fp = os.path.join(dest_csv_dir, ticker_first_letter)
    dest_dir_size = len(os.listdir(dest_csv_fp))

    # extract if overwrite == True or directory is empty
    if (dest_dir_size == 0) or overwrite:
        if os.path.isfile(src_zip_fp):
            print(f"Extracting {src_zip_fp} to {dest_csv_fp}")
        else:
            print(f"File not found: {src_zip_fp}")
            return
        with zipfile.ZipFile(src_zip_fp, 'r') as zip_ref:
            zip_ref.extractall(dest_csv_fp)
    else:
        print(f"{dest_csv_fp} not empty with {dest_dir_size} files")
        return


In [75]:
ZIPS_DIR = os.path.join(FRD_DATA_DIR, asset_type, "zips")

params = {
    "type": "stock",
    "ticker_range": "",
    "timeframe": "1min",
    "adjustment": "adj_splitdiv",
    "period": "full",
    'userid': FRD_USER_ID
}

for ticker_first_letter in tqdm(letters):
    params['ticker_range'] = ticker_first_letter
    print(f"Downloading data for {ticker_first_letter}...")
    download_frd_data(params=params, zips_dir=ZIPS_DIR, overwrite=False)

for letter in tqdm(letters):
    extract_frd_zip(
        src_zips_dir=ZIPS_DIR, 
        dest_csv_dir=CSV_DIR,
        ticker_first_letter=letter,
        period='full',
        adjustment='adj_splitdiv',
        timeframe='1min',
        overwrite=False
    )

  0%|          | 0/26 [00:00<?, ?it/s]